In [1]:
import openslide as os 
from openslide import deepzoom
import os as o 
import numpy as np   
from datetime import datetime

from shutil import copyfile
import xml.etree.ElementTree as ET
import boto3

ModuleNotFoundError: No module named 'openslide'

In [127]:
DIRECTORY_TO_POPULATE = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\'
# Getting the current work directory (cwd)
foldercontents = np.array(o.listdir(DIRECTORY_TO_POPULATE))
contents_unique_sampled = np.array(
    [f.split('.svs')[0] for f in foldercontents[np.flatnonzero(np.core.defchararray.find(foldercontents,'.svs')!=-1)]])
print(contents_unique_sampled)

['42020' '42021' '42024' '42036' '42294' '42522' '46457' '46966' '47104'
 '47148']


In [128]:
DIRECTORY_TO_MINE       = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\'
DIRECTORY_TO_STORE_PATCH   = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\Test_image_Sep26\\'
DEBUG                   = True 
print(DIRECTORY_TO_POPULATE)


C:\Users\Crary\Desktop\FileDumpCode\part_images\


In [123]:
slide_to_annots = {}
print(contents_unique_sampled)
for unique_id in contents_unique_sampled: 
    try: 
        path = DIRECTORY_TO_MINE + str(unique_id) + '.xml'
        slide = os.OpenSlide(path.replace('.xml', '.svs'))
        root = ET.parse(DIRECTORY_TO_MINE + str(unique_id) + '.xml').getroot()
        for annotation in root.iter("Annotation"): 
            region_id = 0 
            total_regions = 0 
            for r in annotation.iter('Vertices'): total_regions += 1 
            
            # Convert type to better suit open_slide expected input 
            readframes = np.array([np.array([float(child.get('X')), float(child.get('Y'))])
                          for child in annotation.iter('Vertex')])
            slide_to_annots[unique_id] = readframes 

            i = 1 
            # Convert the annotated regions to pngs 
            for osf in readframes: 
                print(osf)
                [x, y] = osf 
                loc, level, size = ((int(x-127), int(y-127)), 0, (255, 255))
                img = slide.read_region(loc, level, size)
                file_path_to_save = DIRECTORY_TO_STORE_PATCH  + str(unique_id) + '_' + str(i) + '.png'
                if DEBUG: print(file_path_to_save)
                img.save(file_path_to_save)
                i+=1 

        print("Completed exporting patches for file " + str(unique_id))
    except FileNotFoundError: 
        print("No annotations for file " + str(unique_id))


['44421']
[12196. 15972.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_1.png
[12124. 15975.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_2.png
[12233. 15825.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_3.png
[12321. 15818.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_4.png
[12351. 15772.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_5.png
[12380. 15685.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_6.png
[12486. 15587.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_7.png
[12259. 15521.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_8.png
[12232. 15629.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_9.png
[12177. 15687.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_10.png
[12126. 15675.]
C:\Users\Crary\Desktop\Neuron_classifier\Test_image_Sep26\44421_11.png
[12030. 15793.]
C:\Users\Crary\Desktop\Neu

In [2]:
import numpy as np, cv2, os
import os as o 
#the images are 200x200 pixles, too big for Keras
#resize them to this size
img_size = 60

#-------------get train/test data-----------------
#get data

folders = ['background', 'false_stain', 'part']
labels = []
images = []
folder_path = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\patches\\'
print(folder_path)
for folder in o.listdir(folder_path):
    print(folder)
    for filename in os.listdir(folder_path+folder):
        img = cv2.imread(folder_path+'/'+folder+'/'+filename,0)
        cv2.imshow('img', img)
        #cv2.waitKey(1)
        images.append(cv2.resize(img, (img_size, img_size)))
        #print(folders.index(folder))
        labels.append(folders.index(folder))
        
        
print(images[0])
print("label")
print(labels[0])

C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\
background
false_stain
part
[[205 185 226 ... 179 198 213]
 [156 238 211 ... 230 193 194]
 [228 202 204 ... 155 186 163]
 ...
 [183 187 221 ... 197 203 199]
 [128 239 159 ... 181 178 172]
 [177 104 178 ... 153 191 227]]
label
0


In [6]:
#break data into training and test sets
to_train= 0
train_images, test_images, train_labels, test_labels = [],[],[],[]
for image, label in zip(images, labels):
    if to_train<5:
        #print(to_train)
        #cv2.imshow('img', image)
        train_images.append(image)
        train_labels.append(label)
        to_train+=1
    else:
        #print(to_train)
        test_images.append(image)
        test_labels.append(label)
        to_train = 0
 
print('Number of training images: ', len(train_images))
print('Number of testimg images: ', len(test_images))

Number of training images:  223
Number of testimg images:  44


In [8]:
#function to flatten data
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(len(images), dimData)
    images = images.astype('float32')
    images /=255
    return images


from keras.utils import to_categorical
#flatten data
dataDim = np.prod(images[0].shape)
train_data  = flatten(dataDim, train_images)
test_data = flatten(dataDim, test_images)

#change labels to categorical
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

#determine the number of classes
classes = np.unique(train_labels)
nClasses  = len(classes)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

#three layers
#activation function: both
#neurons: 256
model = Sequential()
model.add(Dense(256, activation = 'tanh', input_shape = (dataDim,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_one_hot, batch_size = 256, epochs=10, verbose=1,
                    validation_data=(test_data, test_labels_one_hot))

#test model
[test_loss, test_acc] = model.evaluate(test_data, test_labels_one_hot)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.python'

In [8]:
from PIL import Image
import  sys


folder = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\test_images\\'
images_df = []
for filename in o.listdir(folder):
        img = Image.open(o.path.join(folder,filename))
        images_df.append(img)

        
from skimage.io import imread_collection

#your path 
col_dir = 'skimage.io\\*.png'

#creating a collection with the available images
Image_collection = imread_collection(col_dir)

for each_file in Image_collection:
    print(Image_collection) 

NameError: name 'o' is not defined

In [64]:
import pyspark 
import pandas as pd 
import pandasql as ps 
from pyspark.sql import Row 
import datetime 
import time 

In [65]:
project_id = 'part'
file_location = 's3:/crary/part'
object_type = 'tangle'

In [66]:
i = 1 
j = 1 
images_rows = {}
annotations_rows = {} 
for sample_number, annotations in slide_to_annots.items(): 
    date_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    row = Row(date_time, sample_number, 1, 's3:/crary/part')
    images_rows[j] = row 
    for annotation in annotations: 
        x, y = annotation 
        annotations_rows[i] = Row(sample_number, j, object_type, x, y, 224, 224, 'dot')
        i += 1 
    j += 1 
    
annots = pd.DataFrame(annotations_rows.values())
imgs_df = pd.DataFrame(images_rows.values())

### Querying Pandas DFs 

In [67]:
ps.sqldf("select * from imgs_df", locals())

,0,1,2,3
0,2019-09-26 14:39:19,42020,1,s3:/crary/part
1,2019-09-26 14:39:19,42021,1,s3:/crary/part
2,2019-09-26 14:39:19,42024,1,s3:/crary/part
3,2019-09-26 14:39:19,42036,1,s3:/crary/part
4,2019-09-26 14:39:19,42294,1,s3:/crary/part
5,2019-09-26 14:39:19,42522,1,s3:/crary/part
6,2019-09-26 14:39:19,46457,1,s3:/crary/part
7,2019-09-26 14:39:19,46966,1,s3:/crary/part
8,2019-09-26 14:39:19,47104,1,s3:/crary/part
9,2019-09-26 14:39:19,47148,1,s3:/crary/part


In [68]:
ps.sqldf(
"select * from annots", locals())

,0,1,2,3,4,5,6,7
0,42020,1,tangle,28250.000000,14975.000000,224,224,dot
1,42020,1,tangle,28138.000000,15003.000000,224,224,dot
2,42020,1,tangle,28470.000000,14969.000000,224,224,dot
3,42020,1,tangle,29097.000000,14798.000000,224,224,dot
4,42020,1,tangle,28426.820238,14718.134833,224,224,dot
5,42020,1,tangle,28394.235968,14676.561799,224,224,dot
6,42020,1,tangle,28396.483159,14841.730341,224,224,dot
7,42020,1,tangle,28007.719108,14701.280900,224,224,dot
8,42020,1,tangle,28108.842705,14728.247193,224,224,dot
9,42020,1,tangle,27861.651690,15014.764051,224,224,dot


### Saving a local file of a Pandas DF 

In [69]:
annots.to_pickle("annotations")
imgs_df.to_pickle("images")


### Saving to S3 

In [80]:
%%bash 
aws s3 cp annotations s3://crary/database/annotations
aws s3 cp images s3://crary/database/images

upload: .\annotations to s3://crary/database/annotations          
upload: .\images to s3://crary/database/images                 


### Downloading from S3 

In [35]:
%%bash 
aws s3 cp s3://crary/database/annotations annotations
aws s3 cp s3://crary/database/images images

download: s3://crary/database/annotations to .\annotations        
download: s3://crary/database/images to .\images               


### Loading the local version of a Pandas DF from a file 

In [36]:
annots = pd.read_pickle("annotations")
imgs = pd.read_pickle("images")

### Appending data to a pandas data frame 

In [37]:
double_imgs = imgs.append(imgs)
ps.sqldf("select * from double_imgs", locals())

,0,1,2,3
0,2019-09-26 11:57:47,42020,1,s3:/crary/part
1,2019-09-26 11:57:47,42021,1,s3:/crary/part
2,2019-09-26 11:57:47,42024,1,s3:/crary/part
3,2019-09-26 11:57:47,42036,1,s3:/crary/part
4,2019-09-26 11:57:47,42294,1,s3:/crary/part
5,2019-09-26 11:57:47,42522,1,s3:/crary/part
6,2019-09-26 11:57:47,46457,1,s3:/crary/part
7,2019-09-26 11:57:47,46966,1,s3:/crary/part
8,2019-09-26 11:57:47,47104,1,s3:/crary/part
9,2019-09-26 11:57:47,47148,1,s3:/crary/part


In [130]:
### Uploading the image file to the right location: 

s3 = boto3.resource('s3')
for unique_id in contents_unique_sampled: 
    path_xml = DIRECTORY_TO_POPULATE + str(unique_id) + '.xml'
    filename_xml=str(unique_id) + '.xml'
    #print(path_xml)
    path_svs = DIRECTORY_TO_POPULATE + str(unique_id) + '.svs'
    filename_svs=str(unique_id) + '.svs'
    s3.Bucket('crary').upload_file(path_xml,'%s/%s' %('part',filename_xml))
    s3.Bucket('crary').upload_file(path_svs,'%s/%s' %('part',filename_svs))
    
  


In [7]:
import os as o
path= 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\Test_image_Sep26\\'
count=0
s3 = boto3.resource('s3')
for filename in o.listdir(path):
    print("file: ", filename)
    file_path= str(path+filename)
    print("file_path: ",file_path)
    count=count+1
    s3.Bucket('crary').upload_file(file_path, '%s/%s' %('part/patch',filename))

print("Finished Uploading o S3 Bucket -Crary")
print("Sarted Uploading o S3 Bucket -background")
    
Path_folder = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\patches\\false_stain\\'
for filename in o.listdir(Path_folder):
    print("file: ", filename)
    file_path= str(Path_folder+filename)
    print("file_path: ",file_path)
    count=count+1
    s3.Bucket('crary').upload_file(file_path, '%s/%s' %('patches/false_stain',filename))

file:  42020_1.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_1.png
file:  42020_10.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_10.png
file:  42020_100.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_100.png
file:  42020_101.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_101.png
file:  42020_102.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_102.png
file:  42020_103.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_103.png
file:  42020_104.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_104.png
file:  42020_105.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_105.png
file:  42020_106.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_106.pn

file:  42020_167.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_167.png
file:  42020_168.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_168.png
file:  42020_169.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_169.png
file:  42020_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_17.png
file:  42020_170.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_170.png
file:  42020_171.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_171.png
file:  42020_172.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_172.png
file:  42020_173.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_173.png
file:  42020_174.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_17

file:  42020_234.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_234.png
file:  42020_235.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_235.png
file:  42020_236.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_236.png
file:  42020_237.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_237.png
file:  42020_238.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_238.png
file:  42020_239.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_239.png
file:  42020_24.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_24.png
file:  42020_240.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_240.png
file:  42020_241.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_24

file:  42020_301.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_301.png
file:  42020_302.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_302.png
file:  42020_303.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_303.png
file:  42020_304.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_304.png
file:  42020_305.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_305.png
file:  42020_306.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_306.png
file:  42020_307.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_307.png
file:  42020_308.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_308.png
file:  42020_309.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_

file:  42020_369.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_369.png
file:  42020_37.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_37.png
file:  42020_370.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_370.png
file:  42020_371.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_371.png
file:  42020_372.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_372.png
file:  42020_373.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_373.png
file:  42020_374.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_374.png
file:  42020_375.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_375.png
file:  42020_376.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42020_37

...

file:  42294_16.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_16.png
file:  42294_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_17.png
file:  42294_18.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_18.png
file:  42294_19.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_19.png
file:  42294_2.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_2.png
file:  42294_20.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_20.png
file:  42294_21.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_21.png
file:  42294_22.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_22.png
file:  42294_23.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_23.png
file:  42294

file:  42294_85.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_85.png
file:  42294_86.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_86.png
file:  42294_87.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_87.png
file:  42294_88.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_88.png
file:  42294_89.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_89.png
file:  42294_9.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_9.png
file:  42294_90.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_90.png
file:  42294_91.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_91.png
file:  42294_92.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42294_92.png
file:  42294

file:  42522_151.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_151.png
file:  42522_152.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_152.png
file:  42522_16.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_16.png
file:  42522_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_17.png
file:  42522_18.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_18.png
file:  42522_19.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_19.png
file:  42522_2.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_2.png
file:  42522_20.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_20.png
file:  42522_21.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_21.png
file:  4

file:  42522_82.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_82.png
file:  42522_83.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_83.png
file:  42522_84.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_84.png
file:  42522_85.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_85.png
file:  42522_86.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_86.png
file:  42522_87.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_87.png
file:  42522_88.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_88.png
file:  42522_89.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_89.png
file:  42522_9.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\Test_image_Sep26\42522_9.png
file:  42522

file:  46966_12.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_12.png
file:  46966_13.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_13.png
file:  46966_14.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_14.png
file:  46966_15.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_15.png
file:  46966_16.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_16.png
file:  46966_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_17.png
file:  46966_18.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_18.png
file:  46966_19.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_19.png
file:  46966_2.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\background\46966_

In [10]:
### Uploading the image file to the right location: 

s3 = boto3.resource('s3')
Path_folder = 'C:\\Users\\Crary\\Desktop\\FileDumpCode\\part_images\\patches\\part\\'
for filename in o.listdir(Path_folder):
    print("file: ", filename)
    file_path= str(Path_folder+filename)
    print("file_path: ",file_path)
    count=count+1
    s3.Bucket('crary').upload_file(file_path, '%s/%s' %('patches/part',filename))
print(count)

file:  42522_1.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_1.png
file:  42522_10.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_10.png
file:  42522_11.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_11.png
file:  42522_12.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_12.png
file:  42522_13.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_13.png
file:  42522_14.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_14.png
file:  42522_15.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_15.png
file:  42522_16.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_16.png
file:  42522_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\42522_17.png
file:  42522_18.png
file_path:  C:\Users\Crary\D

file:  47148_14.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_14.png
file:  47148_15.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_15.png
file:  47148_16.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_16.png
file:  47148_17.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_17.png
file:  47148_18.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_18.png
file:  47148_19.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_19.png
file:  47148_2.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_2.png
file:  47148_20.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_20.png
file:  47148_21.png
file_path:  C:\Users\Crary\Desktop\FileDumpCode\part_images\patches\part\47148_21.png
file:  47148_3.png
file_path:  C:\Users\Crary\De